<a href="https://colab.research.google.com/github/chelsk5/UW-LING-573-Group-5/blob/main/LING573_Model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Authors: Nina Koh, Chelsea Kendrick
# Setup: Clone repo & install dependencies

!rm -rf UW-LING-573-Group-5
!git clone https://github.com/chelsk5/UW-LING-573-Group-5.git

!pip install -r UW-LING-573-Group-5/requirements.txt

Cloning into 'UW-LING-573-Group-5'...
remote: Enumerating objects: 451, done.
remote: Counting objects: 100% (451/451), done.
remote: Compressing objects: 100% (392/392), done.
remote: Total 451 (delta 63), reused 398 (delta 29), pack-reused 0 (from 0)
Receiving objects: 100% (451/451), 340.40 KiB | 1.85 MiB/s, done.
Resolving deltas: 100% (63/63), done.


In [ ]:
%cd UW-LING-573-Group-5/
%ls

/content/UW-LING-573-Group-5
Minutes.md  requirements.txt  summaries.json
README.md   rouge_eval.py     tf-idf.py


In [ ]:
# 1. Load data
from datasets import load_dataset
opinosis = load_dataset("kavgan/opinosis", split="train") # HuggingFace hub
dataset = opinosis.train_test_split(test_size=0.2,seed=1) # follow common split ratio
train_data = dataset["train"]
test_data = dataset["test"]

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [ ]:
# 2. Preprocessing
def clean_data(sent_to_sum):
  """Cleans the given review and returns a dictionary
  of aggregated reviews and the first gold summary"""
  reviews = sent_to_sum["review_sents"].strip()
  # Choose the first gold summary
  gold_summary = sent_to_sum["summaries"][0].strip()
  return {
      "reviews": reviews,
      "gold_summary": gold_summary
  }

# Sanity check!
# Apply cleaning to a small sample
data = load_dataset("kavgan/opinosis")["train"]
cleaned_data = data.map(clean_data)

# Print the result
print("Input reviews:")
print(cleaned_data[0]["reviews"])
print("\nTarget summary:")
print(cleaned_data[0]["gold_summary"])

print("\nInput reviews:")
print(cleaned_data[1]["reviews"])
print("\nTarget summary:")
print(cleaned_data[1]["gold_summary"])

Input reviews:
, and is very, very accurate .
 but for the most part, we find that the Garmin software provides accurate directions, whereever we intend to go .
 This function is not accurate if you don't leave it in battery mode say, when you stop at the Cracker Barrell for lunch and to play one of those trangle games with the tees .
 It provides immediate alternatives if the route from the online map program was inaccurate or blocked by an obstacle .
 I've used other GPS units, as well as GPS built into cars   and to this day NOTHING beats the accuracy of a Garmin GPS .
 It got me from point A to point B with 100% accuracy everytime .
 It has yet to disappoint, getting me everywhere with 100% accuracy .
0 out of 5 stars Honest, accurate review, , PLEASE READ !
 Aside from that, every destination I've thrown at has been 100% accurate .
In closing, this is a fantastic GPS with some very nice features and is very accurate in directions .
 Plus, I've always heard that there are  quirks  

In [ ]:
# 3. Load tokenizer & model
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

model_name = "t5-small"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)

model.safetensors:   0%|          | 0.00/242M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

In [ ]:
# 4. Tokenize reviews & gold summaries
def tokenize_data(cleaned_data):
  """Convert the raw text into token IDs for model processing"""
  # Prepend task for T5 to execute
  pref_text = "summarize: " + cleaned_data["reviews"]

  # Tokenize input (reviews)
  # Ensure that text does not exceed (is in fact exactly) maximum of 512 tokens
  tokenized_input = tokenizer(pref_text, max_length=512, truncation=True, padding="max_length")

  # Tokenize target output (gold summary)
  gold_summary = cleaned_data["gold_summary"]
  labels = tokenizer([gold_summary], max_length=64, truncation=True) # expects a list

  # Add target labels to the model input
  # Format for HuggingFace Trainer
  tokenized_input["labels"] = labels["input_ids"][0] # get first (& only) item of nested list
  return tokenized_input

#apply tokenization to data
tokenized_train = cleaned_data.map(tokenize_data, batched=False) # one row at a time

#check tokenization
print("Tokenized input example (input_ids):")
print(tokenized_train[0]["input_ids"])
print("Tokenized target (labels):")
print(tokenized_train[0]["labels"])



Map:   0%|          | 0/51 [00:00<?, ? examples/s]

Tokenized input example (input_ids):
[21603, 10, 3, 6, 11, 19, 182, 6, 182, 4034, 3, 5, 68, 21, 8, 167, 294, 6, 62, 253, 24, 8, 3121, 1109, 889, 795, 4034, 7943, 6, 213, 3258, 62, 8286, 12, 281, 3, 5, 100, 1681, 19, 59, 4034, 3, 99, 25, 278, 31, 17, 1175, 34, 16, 3322, 2175, 497, 6, 116, 25, 1190, 44, 8, 23291, 49, 13384, 3820, 21, 3074, 11, 12, 577, 80, 13, 273, 3, 11665, 3537, 1031, 28, 8, 3, 17, 15, 15, 7, 3, 5, 94, 795, 5299, 10336, 3, 99, 8, 2981, 45, 8, 367, 2828, 478, 47, 27801, 42, 12468, 57, 46, 22820, 3, 5, 27, 31, 162, 261, 119, 9679, 3173, 6, 38, 168, 38, 9679, 1192, 139, 2948, 11, 12, 48, 239, 4486, 566, 2365, 3853, 7, 8, 7452, 13, 3, 9, 3121, 1109, 9679, 3, 5, 94, 530, 140, 45, 500, 71, 12, 500, 272, 28, 2349, 7452, 334, 715, 3, 5, 94, 65, 780, 12, 26963, 6, 652, 140, 6531, 28, 2349, 7452, 3, 5, 3, 632, 91, 13, 305, 4811, 11772, 222, 6, 4034, 1132, 6, 3, 6, 26573, 391, 19552, 3, 55, 71, 1583, 45, 24, 6, 334, 3954, 27, 31, 162, 3, 12618, 44, 65, 118, 2349, 4034, 3, 5, 86, 

In [ ]:
# 5. Create batch of examples
from transformers import DataCollatorForSeq2Seq

# Dynamically pad sentences to the longest length
data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, model=model)

In [ ]:
# 6. Evaluate model using ROUGE
import evaluate, numpy as np

rouge = evaluate.load("rouge")

def compute_rouge(eval_pred):
  """Decodes token IDs and computes the ROUGE score of generated summaries"""
  predictions, labels = eval_pred # model outputs & gold summaries as token IDs
  # Decode model output IDs back into text
  decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)

  # Replace padding tokens IDs (-100) with tokenizer's pad token ID (usually 0)
  labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
  # Decode gold summary IDs back into text
  decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

  # Compare model output against gold labels via ROUGE
  result = rouge.compute(predictions=decoded_preds, references=decoded_labels, use_stemmer=True)
  clean_result = {k: round(v, 4) for k,v in result.items()} # map rouge type to value
  return clean_result